In [1]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import ctypes
 
################################################
# PLUS 공통 OBJECT
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
 
 
################################################
# PLUS 실행 기본 체크 함수
def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # # 주문 관련 초기화 - 계좌 관련 코드가 있을 때만 사용
    # if (g_objCpTrade.TradeInit(0) != 0):
    #     print("주문 초기화 실패")
    #     return False
 
    return True
 

In [2]:
 
class CpMarketEye:
    def __init__(self):
        self.objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        self.RpFiledIndex = 0
 
 
    def Request(self, codes, dataInfo):
        # 0: 종목코드 4: 현재가 20: 상장주식수
        rqField = [0, 4, 20]  # 요청 필드
 
        self.objRq.SetInputValue(0, rqField)  # 요청 필드
        self.objRq.SetInputValue(1, codes)  # 종목코드 or 종목코드 리스트
        self.objRq.BlockRequest()
 
        # 현재가 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        print("통신상태", rqStatus, self.objRq.GetDibMsg1())
        if rqStatus != 0:
            return False
 
        cnt = self.objRq.GetHeaderValue(2)
 
        for i in range(cnt):
            code = self.objRq.GetDataValue(0, i)  # 코드
            cur = self.objRq.GetDataValue(1, i)  # 종가
            listedStock = self.objRq.GetDataValue(2, i)  # 상장주식수
 
            maketAmt = listedStock * cur
            if g_objCodeMgr.IsBigListingStock(code) :
                maketAmt *= 1000
#            print(code, maketAmt)
 
            # key(종목코드) = tuple(상장주식수, 시가총액)
            dataInfo[code] = (listedStock, maketAmt)
 
        return True
 
class CMarketTotal():
    def __init__(self):
        self.dataInfo = {}
 
 
    def GetAllMarketTotal(self):
        codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
        codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
        allcodelist = codeList + codeList2
        print('전 종목 코드 %d, 거래소 %d, 코스닥 %d' % (len(allcodelist), len(codeList), len(codeList2)))
 
        objMarket = CpMarketEye()
        rqCodeList = []
        for i, code in enumerate(allcodelist):
            rqCodeList.append(code)
            if len(rqCodeList) == 200:
                objMarket.Request(rqCodeList, self.dataInfo)
                rqCodeList = []
                continue
        # end of for
 
        if len(rqCodeList) > 0:
            objMarket.Request(rqCodeList, self.dataInfo)
 
    def PrintMarketTotal(self):
 
        # 시가총액 순으로 소팅
        data2 = sorted(self.dataInfo.items(), key=lambda x: x[1][1], reverse=True)
 
        print('전종목 시가총액 순 조회 (%d 종목)' % (len(data2)))
        for item in data2:
            name = g_objCodeMgr.CodeToName(item[0])
            listed = item[1][0]
            markettot = item[1][1]
            print('%s 상장주식수: %s, 시가총액 %s' %(name, format(listed, ','), format(markettot, ',')))
 

In [3]:
objMarketTotal = CMarketTotal()
objMarketTotal.GetAllMarketTotal()
objMarketTotal.PrintMarketTotal()

전 종목 코드 2967, 거래소 1560, 코스닥 1407
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
전종목 시가총액 순 조회 (2967 종목)
삼성전자 상장주식수: 5,969,782, 시가총액 331,322,901,000,000
SK하이닉스 상장주식수: 728,002,365, 시가총액 68,796,223,492,500
삼성전자우 상장주식수: 822,886,700, 시가총액 37,564,777,855,000
NAVER 상장주식수: 164,813,395, 시가총액 29,913,631,192,500
삼성바이오로직스 상장주식수: 66,165,000, 시가총액 27,921,630,000,000
현대차 상장주식수: 213,668,187, 시가총액 24,785,509,692,000
현대모비스 상장주식수: 95,306,694, 시가총액 23,826,673,500,000
셀트리온 상장주식수: 128,337,853, 시가총액 22,779,968

TIGER 미국채10년선물 상장주식수: 5,700,000, 시가총액 64,923,000,000
모트렉스 상장주식수: 27,250,545, 시가총액 64,856,297,100
세원 상장주식수: 22,382,985, 시가총액 64,798,741,575
삼화네트웍스 상장주식수: 43,047,933, 시가총액 64,787,139,165
흥국에프엔비 상장주식수: 36,706,455, 시가총액 64,603,360,800
KBSTAR 금융채액티브 상장주식수: 640,000, 시가총액 64,576,000,000
EG 상장주식수: 7,500,000, 시가총액 64,425,000,000
이상네트웍스 상장주식수: 9,015,000, 시가총액 64,367,100,000
대구백화점 상장주식수: 10,821,611, 시가총액 64,280,369,340
대창솔루션 상장주식수: 160,428,627, 시가총액 64,171,450,800
비디아이 상장주식수: 7,920,750, 시가총액 64,158,075,000
KODEX 미디어&엔터테인먼트 상장주식수: 5,400,000, 시가총액 64,098,000,000
SG&G 상장주식수: 34,087,196, 시가총액 64,083,928,480
동양피스톤 상장주식수: 13,168,460, 시가총액 64,064,557,900
미래에셋대우우 상장주식수: 14,075,750, 시가총액 63,903,905,000
이라이콤 상장주식수: 12,188,730, 시가총액 63,747,057,900
진양제약 상장주식수: 12,000,000, 시가총액 63,600,000,000
코맥스 상장주식수: 14,561,333, 시가총액 63,560,218,545
YBM넷 상장주식수: 16,312,697, 시가총액 63,537,954,815
한독크린텍 상장주식수: 4,021,000, 시가총액 63,531,800,000
국일신동 상장주식수: 11,090,000, 시가총액 63,213,000,000
잉크테크 상장주식수: 16,019,932, 시가총액 63,198,631,740
우

In [6]:
data2 = sorted(objMarketTotal.dataInfo.items(), key=lambda x: x[1][1], reverse=True)

In [7]:
MarketTotal = []

for item in data2:
    name = g_objCodeMgr.CodeToName(item[0])
    listed = item[1][0]
    markettot = item[1][1]
    
    MarketTotal.append([name, item[0], listed, markettot])

In [8]:
import pandas as pd

In [9]:
format(100000000000, ",")

'100,000,000,000'

In [10]:
format(1000000000000, ",")

'1,000,000,000,000'

In [12]:
pdMarketTotal = pd.DataFrame(MarketTotal)

In [28]:
pdMarketTotal = pdMarketTotal.loc[(
    
    (pdMarketTotal[0].apply(lambda x: x.find("KODEX") < 0)) &
    (pdMarketTotal[0].apply(lambda x: x.find("TIGER") < 0)) &
    (pdMarketTotal[0].apply(lambda x: x.find("KBSTAR") < 0)) &
    (pdMarketTotal[0].apply(lambda x: x.find("HANARO") < 0)) &
    (pdMarketTotal[0].apply(lambda x: x.find("ARIRANG") < 0)) &
    (pdMarketTotal[0].apply(lambda x: x.find("KINDEX") < 0)) &
    (pdMarketTotal[0].apply(lambda x: x.find("KOSEF") < 0)) &
    (pdMarketTotal[0].apply(lambda x: x.find("SMART") < 0)) &
    (pdMarketTotal[0].apply(lambda x: x.find("TREX") < 0)) &
    (pdMarketTotal[0].apply(lambda x: x.find("OTM") < 0)) &
    (pdMarketTotal[0].apply(lambda x: x.find("ATM") < 0)) &
    (pdMarketTotal[0].apply(lambda x: x.find("ETN") < 0)) &
    (pdMarketTotal[0].apply(lambda x: x.find("ETF") < 0)) 
    
)]

In [33]:
pdMarketTotal_200 = pdMarketTotal[:200]

In [34]:
pdMarketTotal_200.columns = "Name,Code,Stock,capitalization".split(",")

In [38]:
list(pdMarketTotal_200["Name"])

['삼성전자',
 'SK하이닉스',
 '삼성전자우',
 'NAVER',
 '삼성바이오로직스',
 '현대차',
 '현대모비스',
 '셀트리온',
 'LG화학',
 'POSCO',
 '삼성물산',
 '신한지주',
 'LG생활건강',
 'KB금융',
 'SK텔레콤',
 '한국전력',
 'SK',
 '기아차',
 '삼성SDI',
 '삼성에스디에스',
 '삼성생명',
 'SK이노베이션',
 '카카오',
 'KT&G',
 '엔씨소프트',
 'LG',
 '아모레퍼시픽',
 'LG전자',
 '삼성화재',
 '하나금융지주',
 'S-Oil',
 '삼성전기',
 '한국조선해양',
 '우리금융지주',
 '고려아연',
 '넷마블',
 '롯데케미칼',
 '셀트리온헬스케어',
 'KT',
 '아모레G',
 '기업은행',
 '웅진코웨이',
 '강원랜드',
 'LG유플러스',
 'LG디스플레이',
 '한온시스템',
 '현대중공업지주',
 '현대글로비스',
 '미래에셋대우',
 'GS',
 '에이치엘비',
 '현대건설',
 '삼성중공업',
 '삼성카드',
 '오리온',
 '현대제철',
 '맥쿼리인프라',
 '한국금융지주',
 '한국타이어앤테크놀로지',
 '롯데지주',
 '삼성엔지니어링',
 '롯데쇼핑',
 '호텔신라',
 'CJ제일제당',
 'DB손해보험',
 'NH투자증권',
 '에스원',
 '한미약품',
 '두산밥캣',
 '한국가스공사',
 '이마트',
 '삼성증권',
 'CJ ENM',
 'CJ대한통운',
 '한국항공우주',
 'LG이노텍',
 '휠라코리아',
 '한화케미칼',
 '대림산업',
 '포스코케미칼',
 'GS리테일',
 '대우조선해양',
 '유한양행',
 'BGF리테일',
 '신세계',
 '현대차2우B',
 '쌍용양회',
 'CJ',
 '제일기획',
 '대한항공',
 'KCC',
 '한미사이언스',
 'GS건설',
 'BNK금융지주',
 '펄어비스',
 '팬오션',
 '더존비즈온',
 '메리츠종금증권',
 '금호석유',
 '포스코인터내셔널',
 '한진칼',
 '오렌지라이프

In [39]:
pdMarketTotal_200.to_csv("../data/KRX200_LIST.csv", index=None)